In [1]:
import torch
from supervoice.tokenizer import Tokenizer
from supervoice.model import SupervoiceGPT
from train_config import config

In [2]:
# Model
tokenizer = Tokenizer(config, "tokenizer_text.model")
model = SupervoiceGPT(config)
# checkpoint = torch.load(f'./output/gpt_new_tokenizer.pt', map_location="cpu")
# model.load_state_dict(checkpoint['model'])
model.eval()
print('ok')

ok


In [3]:
source = tokenizer.encode("What, time?")

model(input = torch.tensor(source).unsqueeze(0), output = torch.tensor([[0]]))

RuntimeError: output with shape [1, 8, 1, 1] doesn't match the broadcast shape [1, 8, 5, 5]

In [ ]:
# Prepare input
# "He was anxious to make this clear."
# ç iː  w ə z  æ ɲ c ʃ ə s  t ə  m ej k  d̪ ɪ s  c ʎ ɪ ɹ
input = "what?".lower()
expected = ""
tokenized = tokenizer.encode(input)
tokenized = [tokenizer.sequence_begin_token_id, tokenizer.text_begin_token_id] + tokenized + [tokenizer.text_end_token_id, tokenizer.phonemes_begin_token_id]
tokenized = torch.tensor(tokenized)
stop_tokens = [tokenizer.sequence_end_token_id]
max_generate_len = 512

In [ ]:
# Generate
input = tokenized
res = model.generate(input.unsqueeze(0), max_new_tokens = max_generate_len, stop_tokens = stop_tokens, top_k = 6).squeeze(0)
res = res.tolist()
print(res)
print(tokenizer.decode(res).replace("•", "<SIL>"))

In [ ]:
# Generate
# input = torch.tensor([tokenizer.phonemes_begin_token_id])
input = tokenized
total = 16
probs, indices = model.predict_next(input, total, trim_generated = False)
probs = probs.tolist()
for i in range(total):
    ttt = tokenizer.decode([indices.tolist()[i]])
    print(str(indices[i]) + ": " +ttt + ": " + str(probs[i]))

In [ ]:
# Iterative
input = tokenized

In [ ]:
total = 16
probs, indices = model.predict_next(input, total, trim_generated = False)
probs = probs.tolist()
tokens = tokenizer.decode(indices.tolist())
for i in range(total):
    ttt = tokenizer.decode([indices.tolist()[i]])
    print(str(indices[i]) + ": " +ttt + ": " + str(probs[i]))
input = torch.tensor(input.tolist() + [indices[0]]).long()